Die Bedingungen des System sind:

!pip install -U --pre tensorflow=="2.*"
!pip install pycocotools

Da diese jedoch schon installiert sein sollten. Sind die nachher nurnoch für den Docker interessant.
Zudem sollte man den Pythonpath im voraus anpassen, da es ohne häufig zu Fehlern kommt.
Dies kann einfach in der Konsole geschehen

export PYTHONPATH=$PYTHONPATH:/users/henry/projects/Tensorflow/models/researchexport

PYTHONPATH=$PYTHONPATH:/users/henry/projects/Tensorflow/models/research/object_detectionexport

PYTHONPATH=$PYTHONPATH:/users/henry/projects/Tensorflow/models/research/slim


Get tensorflow/models or cd to parent directory of the repository.

In [1]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Klone nach 'models' ...
remote: Enumerating objects: 2626, done.
remote: Counting objects: 100% (2626/2626), done.
remote: Compressing objects: 100% (2293/2293), done.
remote: Total 2626 (delta 520), reused 1311 (delta 299), pack-reused 0
Empfange Objekte: 100% (2626/2626), 32.13 MiB | 2.70 MiB/s, Fertig.
Löse Unterschiede auf: 100% (520/520), Fertig.
Aktualisiere Dateien: 100% (2531/2531), Fertig.


Compile protobufs and install the object_detection package

In [2]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto:5:1: warning: Import object_detection/protos/image_resizer.proto is unused.


In [3]:
%%bash 
cd models/research
pip install .

Processing /Users/henry/Desktop/Tensorflow/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1112541 sha256=45868604746c6547afe016c4f1c4f93847aa19a75fdd3f75d0899137dea4f87e
  Stored in directory: /private/var/folders/dv/ph24zr117659m66xf7kpjfk00000gn/T/pip-ephem-wheel-cache-5hq7hes9/wheels/a2/9f/f2/893fe0964a40a7227cc20e25ee9cb5ba572b6f2dc2b422ac29
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


### Imports

In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

ModuleNotFoundError: No module named 'tf_slim'

Patches:

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Modelvorbereitung

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

# Label Map laden

#### "mscoco_complete_label_map.pbtxt"       ist für COCO Trainierte Modelle
#### "oid_bbox_trainable_label_map.pbtxt"      ist für Open Image v2 Modelle
#### "oid_v4_label_map.pbtxt"                            ist für Open Image v4 Modelle

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/oid_v4_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Laden der Bilder
#### Die Bilder liegen in: research/object_detection/test_images

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

# Erkennung
### Laden des Models
#### Zu finden sind diese hier: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [ ]:
model_name = 'faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12'
detection_model = load_model(model_name)

In [ ]:
print(detection_model.inputs)

In [ ]:
detection_model.output_dtypes

In [ ]:
detection_model.output_shapes

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)